In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
data = pd.read_csv("team_data_collected_df.csv")
data.head()

,Location,Team,Opponent,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score
0,H,DUKE,Maine,0.1,125.3,95.2,130.6,64.3,17.7,35.5,31.7,24-34,11-29,84.3,39.5,19.0,19.4,31.6,18-43,3-14,62,96
1,H,DUKE,Army,0.1,124.7,92.3,141.0,61.3,11.3,43.6,25.4,18-33,17-38,81.8,39.1,18.3,23.3,14.1,13-35,8-29,58,100
2,N,DUKE,Kentucky,-0.2,106.8,86.4,95.7,42.3,9.3,25.0,23.9,24-47,4-24,102.3,47.6,14.6,23.3,38.1,15-38,10-25,77,72
3,H,DUKE,Wofford,-0.1,124.7,56.9,133.7,61.3,15.5,45.2,17.7,14-24,16-38,54.4,28.9,29.5,29.3,5.3,9-24,5-33,35,86
4,A,DUKE,Arizona,0.6,111.7,75.9,101.9,50.0,20.7,35.1,21.3,17-36,9-25,81.2,45.3,22.2,16.7,20.8,15-30,6-23,55,69


In [3]:
data.dtypes

Location           object
Team               object
Opponent           object
WAB               float64
ADJO               object
ADJD               object
EFF               float64
EFG%              float64
TO%               float64
OR%               float64
FTR               float64
2P                 object
3P                 object
Opp EFF           float64
Opp EFG%          float64
Opp TO%           float64
Opp OR%           float64
Opp FTR           float64
Opp 2P             object
Opp 3P             object
Opponent_score      int64
Team_score          int64
dtype: object

In [4]:
data.isnull().sum()

Location          0
Team              0
Opponent          0
WAB               0
ADJO              1
ADJD              1
EFF               0
EFG%              0
TO%               0
OR%               0
FTR               0
2P                0
3P                0
Opp EFF           0
Opp EFG%          0
Opp TO%           0
Opp OR%           0
Opp FTR           0
Opp 2P            0
Opp 3P            0
Opponent_score    0
Team_score        0
dtype: int64

In [5]:
data[data.isnull().any(axis=1)]

,Location,Team,Opponent,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score
265,H,Dayton,Capital,0.4,NaN,NaN,116.7,59.3,18.4,36.4,44.4,20-31,8-23,84.5,40.6,18.4,30.4,9.4,11-32,10-32,55,76


In [6]:
data_clean = data.dropna()

In [7]:
columns_to_convert = [
    "Location",
    "ADJO",
    "ADJD",
    "EFG%",
    "TO%",
    "OR%",
    "FTR",
    "Opp EFG%",
    "Opp TO%",
    "Opp OR%",
    "Opp FTR",
]

for col in columns_to_convert:
    data[col] = pd.to_numeric(data[col], errors="coerce")

In [ ]:
# List of columns to convert (with string ratios)
ratio_columns = ["2P", "3P", "Opp 2P", "Opp 3P"]

# Convert each ratio column to numerical percentages
for column in ratio_columns:
    data[column] = (
        data[column]
        .str.split("-")
        .apply(lambda x: int(x[0]) / int(x[1]) if len(x) == 2 and int(x[1]) != 0 else 0)
    )

In [10]:
categorical_columns = ["Team", "Opponent", "Location"]
data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

data_encoded.head(10)

,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,Team_Arizona,Team_Auburn,Team_Boston College,Team_California,Team_Campbell,Team_Clemson,Team_DUKE,Team_Dayton,Team_Florida St.,Team_Georgia Tech,Team_Houston,Team_Illinois,Team_Kansas,Team_Miami FL,Team_Michigan,Team_N.C.State,Team_Norte Dame,Team_PITT,Team_Purdue,Team_Rider,Team_SMU,Team_Stanford,Team_Syracuse,Team_Tennessee,Team_UCLA,Team_UNC,Team_Virginia,Team_Virginia Tech,Opponent_Alabama,Opponent_Alabama St.,Opponent_Albany,Opponent_American,Opponent_Arizona,Opponent_Arkansas,Opponent_Arkansas St.,Opponent_Army,Opponent_Auburn,Opponent_Austin Peay,Opponent_BYU,Opponent_Ball St.,Opponent_Baylor,Opponent_Bethune Cookman,Opponent_Binghamton,Opponent_Boise St.,Opponent_Boston University,Opponent_Bucknell,Opponent_Buffalo,Opponent_Butler,Opponent_Cal Baptist,Opponent_Cal Poly,Opponent_Cal St. Bakersfield,Opponent_Cal St. Fullerton,Opponent_California,Opponent_Campbell,Opponent_Canisius,Opponent_Capital,Opponent_Central Arkansas,Opponent_Charleston Southern,Opponent_Cincinnati,Opponent_Clemson,Opponent_Cleveland St.,Opponent_Coastal Carolina,Opponent_Colgate,Opponent_Connecticut,Opponent_Coppin St.,Opponent_Cornell,Opponent_Creighton,Opponent_Dartmouth,Opponent_Davidson,Opponent_Dayton,Opponent_Delaware,Opponent_Delaware St.,Opponent_Denver,Opponent_Drake,Opponent_Duke,Opponent_Eastern Illinois,Opponent_Eastern Kentucky,Opponent_Elon,Opponent_Evansville,Opponent_Fairfield,Opponent_Fairleigh Dickinson,Opponent_Florida,Opponent_Florida A&M,Opponent_Florida St.,Opponent_Furman,Opponent_Gardner Webb,Opponent_George Mason,Opponent_Georgetown,Opponent_Georgia,Opponent_Georgia St.,Opponent_Georgia Tech,Opponent_Grand Canyon,Opponent_Green Bay,Opponent_Hawaii,Opponent_Hofstra,Opponent_Holy Cross,Opponent_Houston,Opponent_Howard,Opponent_Idaho St.,Opponent_Illinois,Opponent_Incarnate Word,Opponent_Iowa,Opponent_Iowa St.,Opponent_Jackson St.,Opponent_Jacksonville,Opponent_Kansas,Opponent_Kent St.,Opponent_Kentucky,Opponent_LSU,Opponent_La Salle,Opponent_Le Moyne,Opponent_Lehigh,Opponent_Little Rock,Opponent_Louisiana,Opponent_Louisville,Opponent_Loyola MD,Opponent_Maine,Opponent_Manhattan,Opponent_Marquette,Opponent_Marshall,Opponent_Maryland,Opponent_Maryland Eastern Shore,Opponent_Massachusetts,Opponent_McNeese St.,Opponent_Memphis,Opponent_Mercyhurst,Opponent_Merrimack,Opponent_Miami FL,Opponent_Miami OH,Opponent_Michigan,Opponent_Michigan St.,Opponent_Mississippi,Opponent_Mississippi St.,Opponent_Missouri,Opponent_Missouri St.,Opponent_Montana,Opponent_Morgan St.,Opponent_Murray St.,Opponent_N.C. State,Opponent_Navy,Opponent_New Mexico,Opponent_New Mexico St.,Opponent_Newberry,Opponent_Norfolk St.,Opponent_North Alabama,Opponent_North Carolina,Opponent_North Dakota,Opponent_North Florida,Opponent_Northern Arizona,Opponent_Northern Kentucky,Opponent_Northwestern,Opponent_Northwestern St.,Opponent_Notre Dame,Opponent_Oakland,Opponent_Ohio St.,Opponent_Oklahoma,Opponent_Oklahoma St.,Opponent_Old Dominion,Opponent_Oregon,Opponent_Penn St.,Opponent_Pfeiffer,Opponent_Pittsburgh,Opponent_Prairie View A&M,Opponent_Presbyterian,Opponent_Purdue,Opponent_Quinnipiac,Opponent_Radford,Opponent_Rice,Opponent_Richmond,Opponent_Rider,Opponent_Rutgers,Opponent_SIU Edwardsville,Opponent_SMU,Opponent_Sacramento St.,Opponent_Saint Francis,Opponent_Samford,Opponent_San Diego,Opponent_San Diego St.,Opponent_San Francisco,Opponent_Santa Clara,Opponent_Seattle,Opponent_South Carolina,Opponent_Southern Utah,Opponent_St. John's,Opponent_Stanford,Opponent_Stonehill,Opponent_Stony Brook,Opponent_Syracuse,Opponent_TCU,Opponent_Tarleton St.,Opponent_Temple,Opponent_Tennessee,Opponent_Tennessee Martin,Opponent_Texas,Opponent_Texas A&M,Opponent_Texas A&M Corpus Chris,Opponent_Texas Southern,Opponent_Texas Tech,Opponent_The Citadel,Opponent_Toledo,Opponent_Troy,Opponent_Tulane,Opponent_UC Davis,Opponent_UCLA,Opponent_UNC Asheville,Opponent_UNC G

In [11]:
data_encoded = data_encoded.apply(lambda x: x.astype(int) if x.dtype == "bool" else x)

In [12]:
data_encoded.dtypes

WAB                                float64
ADJO                               float64
ADJD                               float64
EFF                                float64
EFG%                               float64
TO%                                float64
OR%                                float64
FTR                                float64
2P                                 float64
3P                                 float64
Opp EFF                            float64
Opp EFG%                           float64
Opp TO%                            float64
Opp OR%                            float64
Opp FTR                            float64
Opp 2P                             float64
Opp 3P                             float64
Opponent_score                       int64
Team_score                           int64
Team_Arizona                         int64
Team_Auburn                          int64
Team_Boston College                  int64
Team_California                      int64
Team_Campbe

In [13]:
numeric_columns = ["Team_score", "Opponent_score"]

# Initialize scaler
scaler = StandardScaler()

# Scale numeric columns
data_encoded[numeric_columns] = scaler.fit_transform(data_encoded[numeric_columns])

In [14]:
X = data_encoded.drop(columns=["Team_score", "Opponent_score"])
y = data_encoded[["Team_score", "Opponent_score"]]

X = X.fillna(0)

In [15]:
X.dtypes

WAB                                float64
ADJO                               float64
ADJD                               float64
EFF                                float64
EFG%                               float64
TO%                                float64
OR%                                float64
FTR                                float64
2P                                 float64
3P                                 float64
Opp EFF                            float64
Opp EFG%                           float64
Opp TO%                            float64
Opp OR%                            float64
Opp FTR                            float64
Opp 2P                             float64
Opp 3P                             float64
Team_Arizona                         int64
Team_Auburn                          int64
Team_Boston College                  int64
Team_California                      int64
Team_Campbell                        int64
Team_Clemson                         int64
Team_DUKE  

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
model = MLPRegressor(
    hidden_layer_sizes=(32, 16), activation="relu", max_iter=500, random_state=42
)

In [19]:
model.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42)

In [20]:
y_pred = model.predict(X_test)

In [21]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared: {r2}")

Mean Absolute Error (MAE): 0.60
Mean Squared Error (MSE): 0.53
R-squared: 0.46266962563534564
